# Автоенкодеры (AE). Вариационные автоэнкодеры (VAE). Условные вариационные автоенкодеры (CVAE). 


## Unsupervised learning 

*If intelligence was a cake, unsupervised learning would be the cake, supervised learning would be the icing on the cake, and reinforcement learning would be the cherry on the cake.* — **Yann LeCun**

Сегодня с вами мы разберем подход, который относится к задаче обучения без учителя (**unsupervised learning**), когда объекты известны, но им не сопоставлены метки, которые мы должны тем или иным способом предсказывать.

Разницу между двумя задачами можно понять при помощи картинки, представленной ниже. 

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/uns_sup_diff.png" alt="alttext" style="width: 700px;"/>

В случае **supervised learning** для каждого объекта нам известна метка, что вот это — изображение яблок, это — изображение груш и т.д. Далее модель учится по изображению определять фрукт. 

В случае же **unsupervised learning** модель просматривает все изображения фруктов, не зная, где какой фрукт находится, и далее формирует представление, которое **неявно** делит фрукты по похожести. 

Зачем вообще изучать такой тип задачи? 

1. Иногда у нас слишком мало размеченных объектов, чтобы учить на них какую-либо задачу классификации и т. д. При этом у нас огромное количество неразмеченных данных. Мы можем **надеяться**, что если мы как-то обработаем наши данные, то они сами разделятся каким-то образом, согласующимся с метками. 

2. Человеческий мозг в основном учится в unsupervised манере. Возможно, для того чтобы решать задачи, которые легко по сравнению с компьютером решает человек, нам стоит и компьютер учить похожим образом.

3. Часто обучение без учителя дает результаты, которые в дальнейшем позволяют быстро адаптироваться к новым задачам обучения с учителем и переключаться между ними. Причем делать это **эффективнее**, чем transfer learning. (*Stop learning tasks, start learning skills* — Satinder Singh).

## Representation learning

Одной из областей, тесно связанной с Unsupervised learning, является задача выучивания представления данных (**representation learning**).

Выученное представление используется либо как шаг предобучения нейросети перед обучением ее уже на целевую задачу, либо как источник признаков для других, не обязательно нейросевых, алгоритмов. 

Задачу получения полезного представления данных можно решать при помощи supervised learning, однако, как правило, это куда менее эффективно. Почему?

ImageNet содержит 1.28 миллионов изображений. Допустим даже мы передаем их в сетку, приводя все к разрешение 128x128. В этом случае они будует занимать $\approx$ 500Гбит. 

А сколько будут занимать метки изображений? У нас 1000 возможных классов. Допустим, мы используем one-hot encoding в виде битового вектора. Получается $\approx$ 12.8Мбит.

Нейронная сеть для ImageNet может легко содержать 30М весов.

Есть ли смысл нейросетке учить информации больше, чем закодировано в **предсказываемой величине**? — Нет. Потому, как правило, представление данных, полученное из полностью supervised нейросети, будет крайне бедным и заточенным на конкретную задачу. Могут возникнуть проблемы даже с применением этого представления для похожей задачи. 

А вот если мы воспользуемся unsupervised подходом, то потенциально можем принудить нейросетку пытаться выработать эффективное представление всех 500Гбит.


## Понижения размерности

Еще одной областью, тесно связанной с предыдущими двумя, является задача снижения размерности — когда мы хотим данные из пространства высокой размерности преобразовать в пространство более низкой, с сохранением одного или нескольких свойств, например:

* данные реконструируются обратно почти без ошибки;
* расстояние между объектами сохраняется.

Зачем это нужно? По многим причинам.

Многие алгоритмы показывают себя плохо на простанствах большой размерности в принципе ([проклятье размерности](https://en.wikipedia.org/wiki/Curse_of_dimensionality)). 

Некоторые — просто будут значительно дольше работать, при этом качество их работы не изменится от уменьшения размерности. 

Понижение размерности позволяет использовать память более эффективно и подавать модели на обучение за один раз больше объектов. 

Также понижение размерности помогает избавится от шума, как мы обсудим дальше. 

## Автоэнкодер

Autoencoder — архитектура нейросети, которая сначала с помощью нейросети-энкодера сжимает изображение в вектор небольшой размерности (он называется скрытым представлением), а затем восстанавливает этот вектор в исходную картинку с помощью нейросети-декодера. 


Практика показывает, что скрытое представление картинки позволяет делать очень интересные и красивые вещи — например, очищать изображение от шума, проводить гладкую интерполяцию между 2 написанными от руки цифрами, генерировать новую рукописную цифру со стилем от имеющейся. 

Откуда берутся эти свойства? Они являются следствием сжатия информации. Одна из форм сжатия — это классификация, которую мы уже делали. Если это цифры, то вместо изображения можно сохранить только какая это цифра. Это предельное сжатие информации, но при попытке перевести цифру в картинку мы уже не имеем достаточно информации, чтобы картинка получалось разной. Если не так сильно ограничивать информацию в точке максимального сжатия, то кроме класса цифры сохранится еще что-то и изображение удастся восстановить с большим количеством сохранённых деталей.


![alt text](https://edunet.kea.su/repo/src/L0X_Encoders/img/better_enc_dec.png)

## Сжатие информации и потери

Автоэнкодер может быть без потерь и с потерями (lossless и lossy). В какой-то степени это альтернативно методам сжатия архиваторов и кодирования контента (zip, mp3, jpeg, flac, ...). Можно ли сделать сжатие на нейронных сетях с помощью автоэнкодеров? Да, это будет работать. Размер сети будет большим, но сжатие может превзойти другие алгоритмы. Исследования в этой области ведутся, но практически применямых примеров нет. 

![alt text](https://edunet.kea.su/repo/src/L0X_Encoders/img/lossy_encoding.png)

Почему это может работь? Дело в том, что нейронная сеть может сформулировать набор правил, по которому на основе латентного представления приближенно или точно кодировать (за счет кодировщика), а затем восстанавливать исходный ообъект (при помощи декодировщика).


<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/ae_principle.png" alt="alttext" style="width: 700px;"/>
А почему мы уверены, что такой набор правил будет существовать и мы вообще имеем право понижать размерность пространства?


## Manifold assumption 

Мы предполагаем, что наши данные на самом деле лежат в пространстве меньшем, чем пространство исходных признаков.

![alt text](https://edunet.kea.su/repo/src/L0X_Encoders/img/manifold1.png)

В большинстве случаев это действительно правда. Например, лица людей даже на фотографиях 300x300, очевидно, лежат в пространстве меньшей размерности, нежели 90000. Ведь не каждая матрица 300 на 300, заполненная какими-то значениями от 0 до 1, даст нам изображение человека

![alt text](https://edunet.kea.su/repo/src/L0X_Encoders/img/manifold2.png)


## Метод главных компонент (PCA). 
Метод главных компонент (Principal Component Analysis). Это метод отображения векторов свойств объектов (помним, что у нас объект всегда описывается вектором свойств, длина вектора — это количество свойств) в вектора производных свойств (**компонент**), меньшей длины с помощью линейной комбинации, чтобы обратной операцией можно было восстановить значения векторов свойств как можно ближе к исходным. То есть PCA тоже выполняет сжатие информации, он тоже работает для группы объектов (а нейронная сеть автоэнкодера учится под определённую группу объектов). 

Обычно PCA работает для центрированных переменных. Каждая следующая компонента проводится перпендикулярно предыдущим и так, чтобы объяснить наибольшую часть разницы, не объясненной предыдущими компонентами между объектами.


<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/pca_motivation.png" alt="alttext" style="width: 700px;"/>

Графически можно представить PCA, как поиск подпространства, проекция точек, на которое минимально меняет координаты в исходном пространстве. Например, для объектов на плоскости PCA можно сделать в одномерное пространство — на прямую.
![alt text](https://edunet.kea.su/repo/src/L0X_Encoders/img/pca_decomposition.png)
Прямая определяется только вектором нормали, то есть линия проекции проходит через 0. 


Возвращаясь к примеру с лицами — долгое время для распознавания лиц размерности 128\*128 использовалось представление, полученное при помощи PCA. Для хорошего качества восстановления хватает около 100 компонент.

## Аналогия AE и PCA

Отличия PCA и AE в том, что PCA выполняет линейную комбинацию над компонентами исходного вектора свойств объекта, а AE — как правило, нелинейную. PCA вычисляется однозначно, а AE обучается без гарантии нахождения наилучшего положения. PCA гарантирует ортогональный базис для разложения сжатых свойств, а AE — нет.

PCA будет частным случаем AE, если в нём сделать только один плотный слой (Dense) с количеством нейронов, равным требуемому числу компонент, сделать линейную функцию активации, сделать потери по среднему квадрату ошибки (mean squared error — MSE). Кроме этого, необходимо будет нормировать признаки перед подачей их на вход AE.

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/pca_autoencoder.png" alt="alttext" style="width: 500px;"/>

Тогда PCA позволяет рассчитать веса для нейронов такого автоэнкодера. При этом гарантировав (в отличии от градиентного спуска) наилучшее решение задачи.

## Очищение изображения от шумов
Интересное применение Autoencoder'ов — очищение входной картинки от шумов. Такое принципиально возможно из-за того, что размерность латентного пространства очень мала по сравнению с размерностью входного пространства(в нашем случае — 32 и 784 соответственно) — в нём попросту нет места случайному шуму, но зато есть место для общих закономерностей из входного пространства.

То есть мы подаём на обучении автоэнкодера такой незашумлённый датасет, что в нём на самом деле есть некое простраство свойств, которое его описывает. На выходе энкодера в изображении останутся именно эти свойства. Шум является внешним свойством и не сможет закодироваться.

Иными словами — за счет кодировщика и декодировщика автоенкодер выучивается «проектировать» объекты на латентное пространство и восстанавливать их из него. Если шум небольшой, то автоенкодер спроецирует объект в нужное место в латентном пространстве и обратно восстановит его уже без шума.

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/denoising_prop.png" alt="alttext" style="width: 700px;"/>


При этом важно понимать, что если шум поместит наш объект так, что автоенкодеру придется выбирать между разными вариантами проекции, могут возникнуть артефакты. 

В случае, приведенном на рисунке, зашумленному x соответствуют две группы объектов из реального датасета. Если мы, к примеру, оптимизируем MSE, то автоенкодеру «экономнее» всего будет восстанавливать нечто между двумя группами. При этом этого «нечто» в природе не существует или оно очень маловероятно. 

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/bad_bimodal.png" alt="alttext" style="width: 400px;"/>


### Добавление шума к исходной выборке

Также в случае отсутствия шума в изначальной выборке, ее малом размере и т.д. можно добавлять шум к самим исходным данным, получая из объекта $x$ объект $\tilde{x}$, и требуя от энкодера восстановить на основе зашумленного объекта исходный. 

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/denoising_autoencoder.png" alt="alttext" style="height: 450px;"/>

Этот подход может работать и является примером *аугментации данных*. Он может дополнительно заставить автоенкодер выучивать полезные признаки, т.е. его можно использовать, даже если целью не является получение автоенкодера, избавляющего данные от шума.

С ним, однако, надо быть очень аккуратным:

1. Шум, который вы добавляете, не должен сильно менять исходный объект. Если это происходит, то либо аввтоенкодер легко будет находить места, где был добавлен шум и при этом делать ему это будет легче, чем учить сжатое представление данных. Либо автоенкодер выучит о ваших данных что-то такое, чего там и в помине быть не может. К примеру, если добавить к признакам, которые всегда целые, нормальный шум, ничего хорошего не выйдет. 


2. Шум должен соответствовать «натуральному шуму». Если реальный шум в данных отличается от того, на котором учился автоенкодер, есть вероятность, что он не будет очищать данные от исходного шума.

## PCA для избавления от шума
Давайте применим PCA, как простеший автоэнкодер, для очищения от шумов изображений базы MNIST. Нам потребуется база MNIST, numpy, библиотека отрисовки matplotlib и сам PCA, который есть в пакете sklearn.

In [ ]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dset

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("whitegrid")

from sklearn.decomposition import PCA
from itertools import chain

USE_CUDA = True

In [ ]:
root = './data'

train_set = dset.MNIST(root=root, 
                       train=True, 
                       transform=torchvision.transforms.ToTensor(),
                       download=True)
test_set = dset.MNIST(root=root, 
                      train=False,
                      transform=torchvision.transforms.ToTensor(),
                      download=True)


Загрузим данные и приведём размерность к двумерной, чтобы это был набор векторов свойств.

In [ ]:
x_train = train_set.data.numpy()
y_train = train_set.targets.numpy()
x_test =  test_set.data.numpy()
y_test = test_set.targets.numpy()

x_train, x_test = x_train/255., x_test/255.
xt_shape = x_train.shape
print("Initial shape ", xt_shape)
xt_flat = x_train.reshape(-1, xt_shape[1]*xt_shape[2])
print("Reshaped to ", xt_flat.shape)

Теперь заведём класс PCA и настроим его, чтобы сохранял 90% исходной картинки. Обучим его и посмотрим, сколько ему потребовалось свойств для описания каждой картинки.

In [ ]:
pca = PCA(.90)
xt_encoded = pca.fit_transform(xt_flat)
print("Encoded features ", pca.n_components_)

Энкодер (он же декодер, ведь это просто обратная матрица от энкодера PCA) обучен. Теперь можно проверить, как он закодирует и раскодирует тестовую выборку. Для этого проведём такие же преобразования размерности для неё.

In [ ]:
xtest_shape = x_test.shape
xtest_flat = x_test.reshape(-1, xtest_shape[1]*xtest_shape[2])
xtest_encoded = pca.transform(xtest_flat)
xtest_decoded = pca.inverse_transform(xtest_encoded).reshape(xtest_shape)
print("Decoded xtest_decoded shape is ", xtest_decoded.shape)

Теперь нужно определить функцию для отрисовки изображений MNIST. Она будет выводить несколько изображений в ряд, поэтому будет принимать трёхмерный массив. Шкала не должна быть автоподстраиваемой, так как после обработки изображения выйдут за диапазон (0,1), в котором заданы исходные изображения. Мы зафиксируем шкалу в диапазоне (0,1).

In [ ]:
def plot_images(images, title):
    fig=plt.figure(figsize=(16, 3))
    columns = images.shape[0]
    rows = 1
    for i in range(columns):
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(images[i], cmap='gray_r', clim=(0,1))
    fig.suptitle(title)
    plt.show()

Выведем исходное и закодированное изображение для некоторых изображений, которые мы элегантно случайным образом выберем из всей тестовой выборки.

In [ ]:
sample_indices = np.random.choice(x_test.shape[0], 6)
samples_orig = x_test[sample_indices]
samples_decoded = xtest_decoded[sample_indices]
plot_images(samples_orig, "Original x_test")
plot_images(samples_decoded, "PCA decoded x_test")

Видно, что `pca.n_components_` (87 для 90% PCA) достаточно для описания картинок MNIST вместо 784 исходных пикселей. Но при этом нужно хранить матрицу кодирования-декодирования, а также изображения получаются немного зашумлёнными. Мы получили способ сжатия с потерями для рукописных цифр, где изображение центрировано и отмасштабировано по рамке из 28х28 пикселей (подробней смотрите правила базы MNIST). 

Степень сжатия у нас условно 87/784 ~= 0.11. То есть сжатие в 9 раз. «Условно», так как сжатое изображение хранится во float, а исходное в uint8, который требует в 4 раза меньше байт. 

Теперь посмотрим, как наш автоэнкодер без нейросетей справится с очисткой от зашумления. Для этого сделаем функцию добавления шумов к MNIST выборке и посмотрим результат.

In [ ]:
def mnist_add_noise(noise_factor, dataset):
    return dataset + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=dataset.shape) 

x_test_noisy = mnist_add_noise(0.3, x_test)
samples_noisy = x_test_noisy[sample_indices]
plot_images(samples_noisy, "x_test with added noise")

Теперь нужно провести ту же операцию PCA энкодера и декодера, что выше.

In [ ]:
def PCArecode(dataset):
    dataset_flat = dataset.reshape(-1, dataset.shape[1]*dataset.shape[2])
    return pca.inverse_transform(pca.transform(dataset_flat)).reshape(dataset.shape)

x_filtered = PCArecode(x_test_noisy)
samples_filtered = x_filtered[sample_indices]
plot_images(samples_filtered, "Noise filtered x_test")

Для простоты сравнения напишем функцию, которая будет строить зашумленные и восстановленные образцы друг под другом.

In [ ]:
def plot_digits(*args, invert_colors=True, digit_size = 28, name=None):
    args = [x.squeeze() for x in args]
    n = min([x.shape[0] for x in args])
    figure = np.zeros((digit_size * len(args), digit_size * n))

    for i in range(n):
        for j in range(len(args)):
            figure[j * digit_size: (j + 1) * digit_size,
                   i * digit_size: (i + 1) * digit_size] = args[j][i].squeeze()

    if invert_colors:
        figure = 1-figure

    plt.figure(figsize=(2*n, 
                        2*len(args))
              )
    
    plt.imshow(figure,
               cmap='Greys_r',
               clim=(0,1))
    
    plt.grid(False)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if name is not None:
        plt.savefig(name)
    plt.show()

In [ ]:
plot_digits(samples_noisy, samples_filtered)

Итак, шумы стали значительно меньше, но и артефакты вокруг цифры усилились. Это неудивительно, ведь мы сжимали информацию линейным образом на целых 87 компоненты. Это не позволяет ни глубокого обучения, ни достаточного сжатия. Повышение уровня сжатия приведёт к еще большему количеству артефактов.

### Латентное представление для цифр после PCA 

Посмотрим теперь на то, как делятся наши картинки в латентном представлении. 

In [ ]:
def pca_latent(dataset):
    dataset_flat = dataset.reshape(-1, dataset.shape[1]*dataset.shape[2])
    return pca.transform(dataset_flat)

In [ ]:
def plot_manifold(latent_r, labels, alpha=0.5):
    plt.figure(figsize=(10,10))
    plt.scatter(latent_r[:, 0], latent_r[:, 1], c=labels, cmap="tab10", alpha=0.5)
    plt.colorbar()
    plt.show()

In [ ]:
latent_r = pca_latent(x_test)
plot_manifold(latent_r, y_test)

Видим, что латентное представление слабо разделяет картинки по тому, какие цифры на них изображены. Только 1 расположены более-менее обособленно.

## Реализация автоенкодера
Итак, вспомним, что в автоэнкодере одна сеть переводит пространство свойств в пространство меньшей размерности, а другая сеть восстанавливает исходное изображение. Вместо вычисления коэффициентов сети мы будем её обучать. Для обучения нужно определить функцию потерь. Мы возьмём среднеквадратичное расстояние (MSE). То есть мы требуем, чтобы значения пикселей исходного изображения и восстановленного отличались несильно.
![alt_text](https://edunet.kea.su/repo/src/L0X_Encoders/img/encoder_loss.png)
Мы можем использовать любую сеть для энкодера и декодера: на плотных слоя или на свёрточных.

Теперь нужно задать архитектуру модели. Мы будем использовать последовательную модель и свёрточную  архитектуру. В конце кодировщика должен быть вектор длины `latent_size`. И декодировщик должен принимать этот вектор и восстанавливать до целого изображения.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_size):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=32,
                               kernel_size=(3,3), 
                               padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, 
                               out_channels=32,
                               kernel_size=(3,3),
                               padding=1)
        self.conv3 = nn.Conv2d(in_channels=32,
                               out_channels=64,
                               kernel_size=(3,3), 
                               padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, 
                               out_channels=64,
                               kernel_size=(3,3), 
                               padding=1)
        self.linear = nn.Linear(in_features=3136, 
                                out_features=latent_size)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.conv2(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = F.max_pool2d(x, kernel_size=(2,2))
        x = self.conv3(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.conv4(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = F.max_pool2d(x, kernel_size=(2,2))
        x = x.flatten(start_dim=1)
        x = self.linear(x)
        return x
        
        
        

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_size):
        super().__init__()
        self.linear = nn.Linear(in_features=latent_size, 
                                out_features=1024)
        self.conv1 = nn.Conv2d(64, 64, (3,3), padding=1)
        self.conv2 = nn.Conv2d(64, 64, (3,3), padding=1)
        self.conv3 = nn.Conv2d(64, 32, (3,3), padding=1)
        self.conv4 = nn.Conv2d(32, 1, (3,3), padding=1)

        
    def forward(self, x):
        x = self.linear(x)
        x = x.view(-1, 64, 4, 4)
        x = F.interpolate(x, size=(14, 14))
        x = self.conv1(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.conv2(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = F.interpolate(x, size=(28, 28))
        x = self.conv3(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.conv4(x)
        x = torch.sigmoid(x)
        return x
        

Напишем основную функцию для обучения нейросети. single_pass_handler и loss_handler бдут меняться в зависимости от сетки, которую мы обучаем.

In [ ]:
def train(enc, 
          dec,
          loader, 
          optimizer, 
          single_pass_handler, 
          loss_handler,
          epoch, 
          log_interval=500):
    for batch_idx, (data, lab) in enumerate(loader):
        batch_size = data.size(0)
        optimizer.zero_grad()
        if USE_CUDA:
            data = data.cuda()
            lab = lab.cuda()

        latent, output = ae_pass_handler(encoder, decoder, data, lab)

        loss = loss_handler(data, output, latent)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader), loss.item()))
            
def ae_pass_handler(encoder, decoder, data, *args, **kwargs):
    latent = encoder(data)
    recons = decoder(latent)
    return latent, recons

def ae_loss_handler(data, recons, *args, **kwargs):
    return F.binary_cross_entropy(recons, data)

Зададим загрузчики наших данных

In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    train_set, 
    batch_size=batch_size,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size, 
    shuffle=False)

In [ ]:
latent_size = 2
learning_rate = 1e-4
encoder = Encoder(latent_size=latent_size)
decoder = Decoder(latent_size=latent_size)

if USE_CUDA:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

optimizer = optim.Adam(chain(encoder.parameters(),
                            decoder.parameters()
                           ),
                      lr=learning_rate)

In [ ]:
for i in range(1, 11):
    train(enc=encoder, 
      dec=decoder, 
      optimizer=optimizer,
      loader=train_loader,
      epoch=i, 
      single_pass_handler=ae_pass_handler,
      loss_handler=ae_loss_handler,
      log_interval=100)

Напишем функцию, чтобы удобно прогонять датасет через обученную нейросеть

In [ ]:
def run_eval(encoder, 
             decoder, 
             loader, 
             single_pass_handler,
             return_real=True, 
             return_recontr=True,
             return_latent=True,
             return_labels=True):
  
    if return_real:
        real = []
    if return_recontr:
        reconstr = []
    if return_latent:
        latent = []
    if return_labels:
        labels = []
    with torch.no_grad():
        for batch_idx, (data, lab) in enumerate(loader):  
            if return_labels:
                labels.append(lab.numpy())
            if return_real:
                real.append(data.numpy())
            if USE_CUDA:
                data = data.cuda()
                lab = lab.cuda()
            rep, rec = single_pass_handler(encoder, decoder, data, lab)
            if return_latent:
                latent.append(rep.cpu().numpy())
            if return_recontr:
                reconstr.append(rec.cpu().numpy())
    
    result = {}
    if return_real:
        real = np.concatenate(real)
        result['real'] = real.squeeze()
    if return_latent:
        latent = np.concatenate(latent)
        result['latent'] = latent
    if return_recontr:
        reconstr = np.concatenate(reconstr)
        result['reconstr'] = reconstr.squeeze()
    if return_labels:
        labels = np.concatenate(labels)
        result['labels'] = labels
    return result


Сначала оценим то, как ведет себя наш автоенкодер работает в целом

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, ae_pass_handler)

In [ ]:
plot_digits(run_res['real'][0:9], run_res['reconstr'][0:9])

И посмотрим, какое латетное представление он выучил. 

In [ ]:
plot_manifold(run_res['latent'], run_res['labels'])

А теперь обучим автоенкодер с латентным слоем размера 24. И посмотрим, как он будет бороться с шумом

In [ ]:
latent_size = 24
learning_rate = 1e-4
encoder = Encoder(latent_size=latent_size)
decoder = Decoder(latent_size=latent_size)

if USE_CUDA:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

optimizer = optim.Adam(chain(encoder.parameters(),
                            decoder.parameters()
                           ),
                      lr=learning_rate)

for i in range(1, 11):
    train(enc=encoder, 
      dec=decoder, 
      optimizer=optimizer,
      loader=train_loader,
      epoch=i, 
      single_pass_handler=ae_pass_handler,
      loss_handler=ae_loss_handler,
      log_interval=100)

Сделаем dataloader, который добавляет в наш датасет шум автоматически

In [ ]:
class AddGaussianNoise:
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:
test_noise_set = dset.MNIST(root=root, 
                      train=False,
                      transform=torchvision.transforms.Compose([
                          torchvision.transforms.ToTensor(),
                          AddGaussianNoise(0., 0.1)
                      ]),
                      download=True)

test_noised_dataloader = torch.utils.data.DataLoader(
    torch.utils.data.Subset(test_noise_set, list(range(64))),
    batch_size=batch_size, 
    shuffle=False)

In [ ]:
run_res = run_eval(encoder, decoder, test_noised_dataloader, ae_pass_handler)

In [ ]:
plot_digits(run_res['real'][0:9], run_res['reconstr'][0:9])

Качество сжатия мы оценили визуально выше. Если обратить внимание, то исходные картинки даже почистились от мелких шумов и странностей изображения и больше стали похожи на непрерывные линии. Размерность латентного пространства `latent_size` значительно меньше исходного количества свойств (784), поэтому мы получили неплохое сжатие изображения.

## Разреженный автоенкодер

До этого мы говорили о том, что построение эффективного автоенкодера возможно только в случае, когда размер латентного слоя сильно меньше входных слоев. 

Однако, сделать латентный слой «узким местом» сети можно и иначе. 
Это позволяет модели самой выбирать размер латентного представления и использовать для разных групп объектов латентные представления разного размера, что может быть полезно для части задач и позволять получить более полезное внутреннее представление. 

Для того, чтобы добиться этого введем следующее условие: 
**Для каждого объекта x в среднем на латентном слое должна активироваться лишь малая доля нейронов**

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/sparse_autoencoder.png" alt="alttext" style="height: 450px;"/>

Добавить такое требование в модель можно двумя способами.

### L1-регуляризации

#### Напоминание:

Квадрат от значения аргумента называют L2-регуляризацией или гребневой регрессией. У неё нет свойства отбирать одни признаки против других, так как по мере приближения к нулю параболическая зависимость начинает давать мало вклада в потери, а вклад в основном приходит от больших аргументов.

$$L2 =  \alpha \sum_i w_i^2 $$

Модуль от значения аргумента называют L1-регуляризацией или лассо-регрессией. Эта регуляризация обычно довольно агрессивная и в многомерном пространстве приводит к отбору свойств или признаков объекта, обращая малозначимые коэффициенты под регуляризацией в ноль. То есть если надо минимизировать сумму модулей, то лучше удержать какой-то параметр на величину 'a', но свести малозначимый параметр со значением 'a' до нуля (меньше потери по нему уже не сделать). Последний параметр перестанет действовать, но зато более значимый параметр при разумных значения коэффициента регуляризации не пострадает.

$$L1 = \beta\sum_i |w_i| $$

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/losses.gif" alt="alttext" style="width: 600px;"/>

Для нашей задачи потому подойдет именно L1-регуляризация — нам важно, чтобы нейросеть именно **не использовала** часть весов, а не просто использовала много мелких весов, которые на следующих слоях может увеличить. 

$sparse\_l1\_loss = ||x - decoder(encoder(x))||^2 + \beta \sum_i |a_{i}^{latent}|$



Зададим то, как должен выполнятся forward pass для нашего автоенкодера и как должен подсчитываться лосс.

In [ ]:
def sparse_ae_pass_handler(encoder, decoder, data, *args, **kwargs):
    latent = encoder(data)
    recons = decoder(latent)
    return latent, recons

def sparse_ae_loss_handler(data, recons, latent, beta=0.1, *args, **kwargs):
    return F.binary_cross_entropy(recons, data) + F.l1_loss(latent, torch.zeros_like(latent)) * beta



In [ ]:
latent_size = 16 * 16

learning_rate = 1e-4
encoder = Encoder(latent_size=latent_size)
decoder = Decoder(latent_size=latent_size)

if USE_CUDA:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

optimizer = optim.Adam(chain(encoder.parameters(),
                            decoder.parameters()
                           ),
                      lr=learning_rate)

In [ ]:
from functools import partial
for i in range(1, 10):
    train(enc=encoder, 
      dec=decoder, 
      optimizer=optimizer,
      loader=train_loader,
      epoch=i, 
      single_pass_handler=sparse_ae_pass_handler,
      loss_handler=partial(sparse_ae_loss_handler, beta=0.01),
      log_interval=450)

In [ ]:
run_res = run_eval(encoder, decoder, test_noised_dataloader, sparse_ae_pass_handler)

In [ ]:
plot_digits(run_res['real'][0:9], run_res['reconstr'][0:9])

Видим, что разреженный автоенкодер несмотря на большой размер латентного слоя все равно эффективно убирает шум из изображений.

Посмотрим, как активируются нейроны латентного слоя для каждого класса.

In [ ]:
run_res = run_eval(encoder, decoder, test_loader, sparse_ae_pass_handler)

In [ ]:
_, axs = plt.subplots(nrows=2, ncols=5, figsize=(16,5))
for label in range(0, 10):
    figure = run_res['latent'][run_res['labels'] == label].mean(axis=0)
    figure = figure.reshape(16, 16)
    ax = axs[label % 2, label % 5]
    ax.imshow(figure,
            cmap='Greys_r',
            clim=(0,1))
    ax.grid(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Видим, что нейронов активируются в среднем крайне мало, между классами активирующиеся нейроны отличаются.

Этот подход очень просто реализуется, но в то же время не совсем очевидно, как с помощью него задать условие вида: пусть в среднем на латентном слое активируется 1% нейронов. Понятно, что это косвенно задается величиной коэффициента $\beta$, но хотелось бы задавать это в явном виде. 

### Дивергенция Кульбака-Лейблера


Для данной цели используется дивергенция Кульбака-Лейблера, которая считается по формуле: 

$$KL(P||Q) = \int_X p(x)\log \dfrac {p(x)} {q(x)} dx$$

В теории информации p считается целевым распределением, а q — тем, с которым мы его сравниваем. 
Важно понимать, что при этом KL не является мерой расстояния, а именно в общем случае $KL(P||Q) != KL(Q||P)$


[Оказывается](https://math.stackexchange.com/questions/90537/what-is-the-motivation-of-the-kullback-leibler-divergence), в подобных задачах она как правило обеспечивают бОльшую сходимость к требуемому распределениею, нежели та же L1-регуляризация.


![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/kl_plot.png)

Пусть мы хотим, чтобы на каждом слое для данного объекта активировалось в среднем p% нейронов.

В нашем случае для каждой активации i нейрона латентного слоя $a_i^{latent}$ мы можем решить, активирован нейрон или нет. Например, в случае функции ReLU очевидно, что любое положительное значение активации отвечает активированному нейрону. После этого мы можем посчитать для каждого объекта, какая доля нейронов активировалась в его случае. Получим величину $\hat{p}$

Фактически мы сравниваем два бернулиевских распределения — то, которое хотим мы, с параметром p, и то, которое мы наблюдаем — с оценным параметром $\hat{p}$. 

$$KL(P||Q) =  p(x) \log \dfrac {p(x)} {\hat{p}(x)} + (1 - p(x)) \log \dfrac {(1 - p(x))} {1 - \hat{p}(x)} $$

Далее лишь остается просуммировать данный лосс по батчу.

Можно делать и иначе — требовать, чтобы каждый нейрон в среднем активировался в p% случаев. В этом случае на первом шаге мы усредняем не по всему слою, а по батчам. А вот подсчитанный лосс усреднеяем по всем нейронам слоя. 


## Автоэнкодер, как генератор и его ограничения. Плавная интерполяция
У нас уже была система с латентным пространством и возможностью строить по нему объекты — GAN. Значит в случае автоэнкодеров тоже можно подавать случайный вектор на декодер и получать новые объекты. До этого мы же только восстанавливали исходную картинку.
![alt text](https://edunet.kea.su/repo/src/L0X_Encoders/img/encoder_as_generator.png)

Какое значение вектора выбрать? Мы же никак не управляли латентным пространством. Непонятно какие числа подставлять. Поэтому мы можем выбрать промежуточные значения между двумя представлением двух исходных изображений в латентном пространстве и получить плавную интерполяцию между изображениями. Постепенно свойства одного изображения будут исчезать, а появляться свойства другого.

In [ ]:
def interpolation(vec1, vec2, N_inter):
    intermediate_values = np.zeros((0, vec1.shape[0]))
    for i in range(N_inter):
        intermediate = (1-float(i / N_inter))*vec1 + float(i / N_inter)*vec2
        intermediate_values = np.append(intermediate_values, intermediate.reshape(1, -1), axis=0)
    intermediate_values = np.append(intermediate_values, vec2.reshape(1, -1), axis=0)
    return intermediate_values

N_inter = 8

# Take pairs of random images from the test set
encodings = encoder.predict(np.expand_dims(samples_orig, axis = 3))
for i in range(len(sample_indices) - 1):
    vectors = interpolation(encodings[i], encodings[i+1], N_inter)
    images = np.squeeze(decoder.predict(vectors), axis = 3)
    plot_images(images, "Interpolation of %i into %i"%(y_test[sample_indices[i]], y_test[sample_indices[i+1]]))

Если этот файл запущен не на Google Colab, с помощью этого кода можно создать видео, на котором процесс превращения одной цифры в другую будет виден ещё более наглядно. Для этого можно использовать уже известный нам OpenCV. Он умеет делать видеофайлы из массивов чисел.

In [ ]:
import cv2

N_inter = 90
resize_coeff = 10

size = (images.shape[2]*resize_coeff, images.shape[1]*resize_coeff)
out = cv2.VideoWriter('output/interpolation.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size, 0)
for i in range(len(sample_indices) - 1):
    vectors = interpolation(encodings[i], encodings[i+1], N_inter)
    images = np.squeeze(decoder.predict(vectors), axis = 3)
    for i in range(len(images)):
        img = images[i] / np.max(images[i])
        img = (cv2.resize(255*img.reshape(28, 28), size, cv2.INTER_NEAREST))
        out.write(img.astype(np.uint8))
    
out.release()

Так себе интерполяция вышла. Старое изображение затухает, а новое изображение появляется. Хочется, чтобы в промежуточных кадрах не было каки-то непонятных очертаний, а изображение было чем-то промежуточным по смыслу между стартовым и конечным изображением. Причина в том, что в латентном пространстве действительно возникли зоны, которые умеют декодироваться в хорошие изображения. Но никто не сказал, что между этими зонами должно быть что-то адекватное (что мы видели из представления).

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/bad_latent_for_ae.png" alt="alttext" style="width: 600px;"/>
Представим это графически. Пусть наш очень умный, содержащий очень много коэффициентов, энкодер и декодер смог разложить все входные объекты на одной оси (размерность латентного пространства — 1). По сути он каждому входному изображению присвоил номер и по номеру может это изображение вспомнить. То есть автоэнкодер очень переобученный. Тогда если мы возьмём промежуточный номер (пытаемся интерполировать), то какое изображение мы собираемся получить?

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/bad_latent_for_ae2.png)

Если мы хотим, чтобы декодированные промежуточные латентные состояния имели черты близких к ним объектов, то надо притянуть латентные координаты похожих объектов. Например вот так:
![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/good_latent_for_ae.png)

## Вариационный автоэнкодер


### Мотивация 

Хотим вместо представления слева получить представление справа

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/vae_motivation.png)

При этом зоны пересечения должны действительно содержать переходные картины 
![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/vae_ideal.png)

## Неудачная попытка: регуляризация

Можем попробовать заставить наши объекты «лежать» рядом — будем штрафовать латентные представления, которые далеко уходят от начала координат. 

Можем использовать как L1, так и L2 регуляризацию, так и их комбинацию — elastic loss.

Однако, это приведет просто к масштабированию распределения. Нам надо одновременно получить связное латентное представление — чтобы у нас не возникало зон в латентном представлении, которым не соответствует ничего, и при этом представление, в котором цифры будут отделены друг от друга. 


![alt_text](https://edunet.kea.su/repo/src/L0X_Encoders/img/reconstruction_loss_only.png)

Мы имеем ситуацию, как на картинке слева. Переход-интерполяция между объектами проходит через зону отсутствующих в обучении объектов, декодирование которых даст несуществующие в реальности объекты. Нам не удастся погенерировать новые картинки, преобразовывая случайную точку из латентного пространства в случайную картинку. 




## Вероятностный автоэнкодер

При этом постановка задачи с автоенкодером говорит нам, что есть некое пространство меньшей размерности $Z$, которое и обуславливает процесс генерации объектов из $X$. Все остальные различия — следствия случайности — один и тот же человек может по-разному нарисовать цифру 5. 

Будем искать латентное пространство Z, которое удовлетворяет следующему условию:

$$p(x) = \int p(x, z)dz $$

Кроме того, пусть объекты из Z легко генерировать. 

По формуле совместной вероятности:

$$p(x, z) = p(x|z)p(z) $$

Ну, осталось только подобрать такие параметры, чтобы все работало)) 

К сожалению, сделать это в таком виде не получится. Пространство $X$ может быть высокоразмерным. 

Но - мы можем существенно сузить область поиска, ведь каждому $x$ из пространства $X$ соответствует лишь небольшая возможная область в $Z$.

Для этого будем также учить отображение из пространства $X$ в пространство $Z$, т.е, пытаться выучить $p(z|x)$. Назовем функцию, которой будем его приближать $q(z|x)$. 

Что же в случае автоенкодера выполняет роль p(x|z) и q(z|x) ?
Очевидно — кодировщик и декодировщик соответственно. 

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/vae_as_two_functions.png)


Чтобы все получилось, нужно сделать с кодировщиком две вещи. Заметьте, что декодировщик мы оставим без изменений.

### Первая модификация

Пусть наш кодировщик генерирует на основе объекта X вектор средних и вектор стандартных отклонений.

Этих двух векторов хватает нам для того, чтобы задать многомерное нормальное распределение с независимыми компонентами (чтобы матрица ковариаций была диагональной), соответствующее данному объекту.

Чтобы получить латентное представление объекта, отличающегося от X только в силу случайности нам достаточно сгенерировать вектор из нормального распределения с такими параметрами. 

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/vae_structure.png)

Далее мы можем требовать, чтобы из полученного латентного представления декодировшик восстанавливал объект, похожий на исходный. 

Здесь, однако, сразу возникает возникает проблема с тем, что граф вычислений, соответствующий предыдущей структуре не может пропускать градиент — как пропустить градиент через генератор случайного нормального числа? Если считать из определения, то даже малейшему изменению параметра могут соответствовать бесконечные изменения генерируемого числа (нормальное распределение определено на бесконечности). В общем, проблема. 

Но мы можем вспомнить замечательное свойство одномерного нормального распределения:

$$N(\mu,\sigma^2) = N(0,1) * \sigma + \mu$$ 

Выполняется это и для многомерного случая. Потому сделаем следующее — будем генерировать значение из нормального распределения с средними 0 и дисперсиями 1, а затем домножать это на вектор стандартных отклонений и прибавлять вектор средних. Получится вот такое преобразование, которое называется reparametrization trick.

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/reparametrization_trick.png)

В отличии от левого случая, в правом мы спокойно можем пропустить градиент через детерминистичные ноды. 


Но если просто применить такой принцип, то он снова имеет проблему предыдущего детерминистического подхода, так как вероятностное распределение сможет свернуться в дельта-функцию — зачем нейросети мучаться с объектами немного отличающимися от тех, что есть в обучающей выборке и пытаться нормально их восстанавливать, если можно просто начать генерировать стандартные отклонения, близкие к нулю и тем самым получить $\delta$-функцию, которая будет нашему объекту всегда сопоставлять одну точку в латентном представлении. 

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/Dirac_function_approximation.gif)

### Вторая модификация

Поэтому нам надо ввести регуляризацию, требующую от каждого распределения быть близким к нормальному распределению вокруг нуля координат латентного пространства с дисперсией 1 (наше $P(z)$). 

В принципе, нам подойдёт любая адекватная мера расстояния между двумя распределениями. 

### Только KL-дивергенция

Что произойдет, если мы будем использовать такой лосс при обучении модели?

$$ Loss = - KL(Q(z|x)||P(z)) $$

Видим, что мы забываем забываем про декодировщик — он может выдавать все, что угодно. Потому логично ожидать, что обучится только кодировщик и обучится он отражать наши точки в нормальное распределение со средним 0 и дисперсией 1. И все, большего ему в жизни и не надо. Можете проверить это. 

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/pure_kl_loss.png)

### Совмещаем ошибку восстановления и KL-дивергению


Поэтому мы должны сохранить исходный лосс — декодировщик штрафуется за то, что не может нормально реконструировать объект. 

Формально это записывается следующим образом: 

$$ vae\_loss = E_{z \sim Q(z|x)}[logP(x|z)] - KL[Q(z|x)||P(z)]$$

Вторая компонента осталась без изменений, а первая — это красиво записанное требование корректно восстанавливать объекты из обучающей выборки и чтобы при этом объекты, полученные их небольшим изменением за счет случайности также восстанавливались в объекты, близкие к объектам из тренировочной выборки. И удовлетворять этой компоненте лосса мы можем за счет того же лосса, который использовали в обычном автоенкодере. 

Если подставить на место p(z) нормальное распределение, то получим следующее выражение:


$$KL(N(\mu, \sigma) || N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}$$


$$ vae\_loss = ||x - \tilde{x}||^2 + 0.5 * (1 + \log\sigma^2 - \mu^2 - \sigma^2)$$




Учет обеих компонент позволяет нам получить то, что мы хотели — непрерывное простанство, где нет «дыр» в представлении, и при этом близкие по смыслу объекты расположены рядом, а далекие — далеко. 

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/kl_repr_loss.png)

## Почему KL(Q||P)?

Внимательный слушатель может заметить, что почему-то здесь мы используем на KL(P||Q), как в прошлый раз, а KL(Q||P). Одним из объяснений этого является то, что если Q — распределение, которым мы аппроксимируем реальное, то может минимизация KL(P||Q) и KL(Q||P) ведет к разному результирующему Q. 

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/KL_inclusive_exclusive.png)

Оказывается, чаще всего нам предпочтительнее получать первое поведение, так как лучше не генерировать какую-то часть объектов, чем генерировать объекты, настоящая вероятность которых близка к 0 (обратите внимание на область второго графика, где реальное распределение P жмется к 0, а Q при этом наоборот, достигает максимума).

## Проблемы  «ванильного» VAE

Одна из проблем VAE, с которой можно столкнуться, состоит в том, что две компоненты лосса конфликтуют друг с другом. Если будет доминировать KL-loss, то мы получим представление, из которого наши объекты очень плохо восстанавливаются — они раскиданы по представлению, как угодно. 

Если же наоборот, будем доминировать reconstruction loss, то мы получим ситуацию, в которой объекты восстанавливаются нормально, но никакой непрерывностью и не пахло. 

Проблема возникает и с самой KL-дивергенцией, у которой есть ряд существенных недостатков. Есть другие способы оценки близости двух распределений, которые порой дают лучшие результаты. К ним относится дивергенция Йенсена — Шеннона, которую мы вскользь затронем далее и метрика Васерштейна (используется в Wasserstein autoencoders), изучение которой выходит за рамки курса. 

Кроме того, в случае, когда декодировщик содержит значительно больше параметров, нежели кодировщик, может возникать ситуация, при которой сгенерированное латентной представление игнорируется.

## Реализация VAE 

Давайте реализуем новый энкодер и декодер с ручной вставкой репараметризации и ручным циклом обучения. Начнём с 2 раза большего количества выходов у энкодера.

Так как потребуется довольно много ручного кода, то будем двигаться по шагам. Сначала попробуем реализовать ручное обучение, используя только средние значения энкодера и не используя дисперсию и вообще какие-то распределения вероятностей. Для этого воспользуемся `tf.split` и возьмём половину выходов нейронов. Что будет, если взять другую половину?

In [ ]:
# getDecoder and getEncoder methods are already valid

VAE_latent_size = 24

VAE_encoder = getEncoder(VAE_latent_size * 2)
VAE_decoder = getDecoder(VAE_latent_size)

VAE_input = Input(shape=(28, 28, 1)) # Set shape to (28, 28, 1) if autoencoder is convolutional, (784, 1) otherwise
VAE_encoder_output = VAE_encoder(VAE_input)
VAE_fake_generate, _ = tf.split(VAE_encoder_output, num_or_size_splits=2, axis=1)
VAE_fake_output = VAE_decoder(VAE_fake_generate)
fake_VAE = Model(inputs=VAE_input, outputs=VAE_fake_output)
fake_VAE.summary()

## Инструмент GradientTape
Преобразуем нашу выборку в датасет tensorflow. Нам потребуется рассчитывать потери — введём для этого функцию, просто считающую потери по mse, как ранее для автоэнкодера, без какой-то регуляризации.

Для ручного обучения в tensorflow есть инструмент GradientTape — лента записи событий, для которых можно просить выдать градиент по обучающимся параметрам. Он сам разбирается как такой градиент построить. Далее градиент уже передаётся оптимизатору. Тот сам разбирается, как ему применить градиент для решения задачи оптимизации.

In [ ]:
# x_train, y_train etc are already imported, normalized and expanded for convolutional NN

epochs = 10
train_size = len(x_train)
batch_size = 64
test_size = len(x_test)


train_dataset = (tf.data.Dataset.from_tensor_slices(x_train)
                 .shuffle(train_size).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(x_test)
                .shuffle(test_size).batch(batch_size))

def compute_loss(model, x):
  mse = tf.keras.losses.MeanSquaredError()
  return mse(model(x), x)

#@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

Мы импортируем модель расчёта времени, чтобы знать сколько длится эпоха. Эпохи это реализуемый вручную цикл вызова расчёта потерь, градиента, оптимизации. В цикле мы будем брать батч за батчем, считать время на эпоху, дополнительно для проверки обучения будем считать потери на всех батчах тестовой выборки и выводить статистику.

In [ ]:
import time

for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in train_dataset:
    train_step(fake_VAE, train_x, tf.keras.optimizers.Adam(learning_rate=0.001))
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in test_dataset:
    loss(compute_loss(fake_VAE, test_x))
  print('Epoch: {}, Test set loss: {}, time elapse for current epoch: {:.3}'
        .format(epoch, loss.result(), end_time - start_time))

Теперь пришло время проверить, удалось ли нам вручную проводить обучение и обучить не хуже автоматических инструментов. Для этого сделаем проверку, как для автоэнкодера.

In [ ]:
samples_decoded = np.squeeze(fake_VAE.predict(x_test[sample_indices]), axis = 3)
plot_images(samples_orig, "Original x_test")
plot_images(samples_decoded, "Autoencoder decoded x_test")

Мы смогли получить то же, что и раньше с автоэнкодером. Теперь нам нужно добавить вариации и репараметризацию. Меняются 2 вещи: модель и функция потерь.

Меняется модель VAE, так как теперь не половина выходов энкодера — это входы декодера, а все выходы — это параметры для генерации точки в латентном пространстве, а уже она — вход для декодера. Это сложно оформить моделью tensorflow. Для удобной передачи, как раньше в виде объекта, создадим объект VAE для обучения и использования. Этот объект позволит нам получать trainable_variables, которые нужны в обучении.

Для начала покажем как энкодер передаёт сигнал декодеру на необученной сети. Для этого нам не нужна репараметризация, ведь мы не будем считать градиенты или обратное распространение ошибки.
Цепочка расчёта у нас из 3 звеньев: энкодер, генерация точки в латентном пространстве, передача этой точки на декодер. В конце визуализация, что было на входе и что на выходе.

In [ ]:
# getDecoder, getEncoder methods are already valid, and VAE_latent_size already defined

# Rebuild encoder and decoder to loose connection with previous model.
# But they are essentially the same as before.
VAE_encoder = getEncoder(VAE_latent_size * 2)
VAE_decoder = getDecoder(VAE_latent_size)

# Get probability density parameters (interpret them as such)
mean, logvar = tf.split(VAE_encoder(samples_orig), num_or_size_splits=2, axis=1)
# Instantiate encoded state statistically
z = tf.random.normal(shape=mean.shape) * tf.exp(logvar * .5) + mean
# Run decoder with dropping one dimension for visualization
samples_decoded = np.squeeze(VAE_decoder(z), axis = 3)

plot_images(samples_orig, "Original x_test")
plot_images(samples_decoded, "Fresh VAE decoded x_test")

Теперь то же самое в виде класса. Если есть способ из входа сделать выход, то назовём его recode(), а для потерь нужно только добавить MSE. Также добавим репараметризацию, как функцию, для наглядности.

In [ ]:
class VAE_larva(tf.keras.Model):
    """Variational autoencoder larva."""
    
    def __init__(self, latent_dim):
        super(VAE_larva, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = getEncoder(2 * latent_dim)
        self.decoder = getDecoder(latent_dim)
        self.mse = tf.keras.losses.MeanSquaredError()
        
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean
    
    def recode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        z = self.reparameterize(mean, logvar)
        return self.decoder(z)
        
    def compute_loss(self, x):
        return self.mse(x, self.recode(x))

vael = VAE_larva(VAE_latent_size)

Цикл обучения почти не поменялся. Только вызываем мы теперь методы класса, а не внешние функции.

In [ ]:
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = model.compute_loss(x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in train_dataset:
    train_step(vael, train_x, tf.keras.optimizers.Adam(learning_rate=0.001))
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in test_dataset:
    loss(vael.compute_loss(test_x))
  print('Epoch: {}, Test set loss: {}, time elapse for current epoch: {}'
        .format(epoch, -loss.result(), end_time - start_time))

Мы выучили вариационный автоэнкодер, но без регуляризации. Поэтому пока никаких улучшений ожидать не стоит. Не добавлена еще дивергенция KL в функцию потерь. Но снова можно проверить, что он обучился не хуже предыдущих автоэнкодеров.

In [ ]:
samples_decoded = np.squeeze(vael.recode(samples_orig), axis = 3)

plot_images(samples_orig, "Original x_test")
plot_images(samples_decoded, "VAE larva decoded x_test")

In [ ]:
class VAE(tf.keras.Model):
    """Variational autoencoder finally."""
    
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = getEncoder(2 * latent_dim)
        self.decoder = getDecoder(latent_dim)
        self.mse = tf.keras.losses.MeanSquaredError()
        
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean
    
    def recode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        z = self.reparameterize(mean, logvar)
        return self.decoder(z)
        
    def compute_loss(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        z = self.reparameterize(mean, logvar)
        x_out = self.decoder(z)
        logpx_z = -self.mse(x, x_out)
        logpz = self.log_normal_pdf(z, 0., 0.)
        logqz_x = self.log_normal_pdf(z, mean, logvar)
        return -tf.reduce_mean(10*logpx_z + logpqz - logqz_x)
    
    def log_normal_pdf(self, sample, mean, logvar, raxis=1):
        log2pi = tf.math.log(2. * np.pi)
        return tf.reduce_sum(
            -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
            axis=raxis)

    
vae = VAE(VAE_latent_size)

По сути наша новая функция потерь — это потери на некорректное восстановление изображения + потери на неточное попадание `z` в нормальное распределение с центром в нуле и единичной дисперсией — потери на неточное попадание `z` в нормальное распределение со средним и дисперсией, сгенерированных энкодером.

In [ ]:
for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in train_dataset:
    train_step(vae, train_x, tf.keras.optimizers.Adam(learning_rate=0.001))
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in test_dataset:
    loss(vae.compute_loss(test_x))
  print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
        .format(epoch, -loss.result(), end_time - start_time))

Проверим, наконец, что вариационный автоэнкодер работает как автоэнкодер, визуализировав привычным способом перекодированные изображения.

In [ ]:
samples_decoded = np.squeeze(vae.recode(samples_orig), axis = 3)

plot_images(samples_orig, "Original x_test")
plot_images(samples_decoded, "VAE decoded x_test")

Если мы используем размерность латентного пространства 2, то это позволит нам получать распределение классов цифр на плоскости, типа такого:

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/vae_sampling.png" alt="alttext" style="width: 600px;"/>

Это не просто интерполяция по двум направлениям. Тут именно все 10 цифр должны так занять место на плоскости, чтобы плавно перетекать друг в друга.

### Векторная арифметика

В принципе, можно даже в латентном пространстве брать разницу черт написания двух одинаковых цифр, прибавлять к другой цифре, получая в результате цифру, написанную немного по-другому. 

Подробнее: 

У нас есть 1, написанная без наклона и 1, написанная с наклоном. 
И у нас есть 9 без наклона.

Вычитаем из латентного кода 1 с наклоном латентный код единицы без наклона и прибавляем к 9. Если все пройдет хорошо — получим девятку с наклоном. 

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/vector_arithm.png" alt="alttext" style="width: 700px;"/>

Такое можно делать и для других примеров — добавлять людям на изображении очки. 

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/vector_arithm3.png" alt="alttext" style="width: 400px;"/>

или получать нечто среднее между двумя объектами.
<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/vector_arithm2.png" alt="alttext" style="width: 400px;"/>

## Условные автоенкодеры (CAE)

### Мотивация 

Как, используя обычный VAE, сгенерировать картинку с заданной меткой? 

На самом деле, задача нетривиальна. Как вариант, мы можем понять, в какую область латентного пространства VAE отображает все 0 и затем сэмплировать уже из этой области. 

Хорошо, а если мы хотим нарисовать 1 тем же почерком, которым нарисована данная нам тройка? В этом случае классический VAE вообще не получится использовать. 

На самом деле, есть еще одна проблема. Что, если распределение объектов действительно сильно зависит от какой-то дополнительной информации, например, того, какую цифру хотел изобразить человек? Тогда KL-loss будет пытаться «скрестить ежа с ужом» и в результате мы получим очень странное представление и опять же, на границах могут получаться несуществующие в реальности мутанты (если внимательно посмотрите на предыдущую картинку — так и получается). 



Продемонстрировать мы это можем на модельной задаче. Сгенерируем два несвязных набора точек в двумерном пространстве, каждый из которых представляет собой некий паттерн с добавленным шумом. 

### Несвязные компоненты и условный автокодировщик

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/unconnected_repr.png" alt="alttext" style="width: 600px;"/>

Теперь попробуем обучить обычный автоенкодер. 
И посмотрим, что выучил автоенкодер. Видим, что он показывает связь там, где она явно отсутствует. Требование получить одно и то же представление для точек из двух паттернов мешают автоенкодеру нормально выучить эти паттерны — они получаются смазанными или даже неверными.


<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/bad_latent_unconnected.png" alt="alttext" style="width: 600px;"/>


А что будет, если мы будем передавать в кодировщик и в декодировщик метку объекта? 
Тогда окажется, что наш автокодировщик работает в разы лучше:

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/good_latent_unconnected.png" alt="alttext" style="width: 600px;"/>

## Условные вариационные автоенкодеры,   CVAE

Обычные условные автокодировщики применяются редко — они по-прежнему не гарантируют нам связность представления в пределах одной метки. 



Однако, добавление меток в вариационный автокодировщик часто помогает решать уже описанные задачи на хорошем уровне. 

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/cvae.png" alt="alttext" style="width: 600px;"/>

### Генерация заданных цифр из латентного распределения

Ниже приведено то, как хорошо выполняют задачу генерации заданной цифры нейросеть по мере обучения и то, как выглядит латентное представление объектов, относящихся к данной цифре. 

Для 3:

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/gen_cvae_3.gif" alt="alttext" style="width: 400px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/lat_cvae_3.gif" alt="alttext" style="width: 400px;"/> </td>
</tr></table>


Для 7:

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/gen_cvae_7.gif" alt="alttext" style="width: 400px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/lat_cvae_7.gif" alt="alttext" style="width: 400px;"/> </td>
</tr></table>

### Генерация заданных цифр из латентного распределения

Также успешно такая нейросеть справится в задаче, где мы используем латентное представление одной цифры  для того, чтобы сгенерировать цифру с таким же стилем написания. 

Чтобы сделать это, достаточно просто получить латентное представление для 7, а затем передать его в декодер с меткой 3.

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/style_transfer2.png" alt="alttext" style="width: 600px;"/>

Результаты переноса стилей для нескольких разных 7 представлены ниже.

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/style_transfer.png" alt="alttext" style="width: 500px;"/>

## Состязательные автокодировщики (AAE = AE + GAN)

В ходе курса вы уже познакомились с генеративными состязательными сетями. Возникает искушение как-то использовать принципы, лежащие в их основе, для VAE. Действительно, так делают. Такие нейросети называются **adversarial autoencoders**. 

Использовать мы будем дискриминатор. К какой части нейросети мы можем его «приделать»? На самом деле — к любой. Но самое распространенное — а давайте уберем наши мучения с KL-дивергенцией. Пусть теперь дискриминатор будет отличать латентное представление, которое мы сгенерировали от стандартного нормального распределения. Если может отличить хорошо — то штрафуем енкодер за это.
![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/aae.png)

Внезапно, это избавляет нас от необходимости даже делать какие-то дополнительные манипуляции с кодировщиком — нам не нужно теперь, чтобы он выдавал средние и дисперсии, а потом мы использовали их для генерации объектов из реального распределения. Нам достаточно того, что дискриминатор не может отличить латентное представление, которое мы получаем от нормального распределения. 

Более того, оказывается, что AAE может генерировать более «качественные» объекты, нежели ванильный VAE. Можно теоретически показать, что это является следствием того, что он минимизирует не KL-divergence, а более эффективную дивергенцию Йенсена-Шеннона. Однако подробный разбор этого выходит за рамки курса. 

Кроме того, для AAE легче добиться того, чтобы ваше латентное представление выглядело специфичным образом. Просто генерируем выборку из нужного распределения и говорим дискриминатору пытаться отличать сгенерированную выборку от объектов из латентного представления. 

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/different_latent_spaces.png)

И все это без введения дополнительной лосс-функции! Фактически — GAN и есть наша loss-функция. Мы используем одну нейросеть в качестве лосс-функции для обучения другой нейросети. 

## Разделение (disentangling) стиля и метки 

В CVAE мы полагались на то, что если мы передаем нашим нейросетям метку объекта, то на латентном слое эта метка убирается.
Что ж, это существенное допущение. Если этого не произойдет, то в нашей сгенерированной 7 будет немного 5 и так далее.

А давайте просто сделаем так, чтобы метка отделялась в латентном слое в явном виде. И будем за неверное предсказание метки нейросеть штрафовать. 

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/disentangle_aae.png)



Что нам это дает? — теперь мы можем использовать стиль объекта отдельно и для простых ситуаций быть уверены, что в стиль объекта не замешалась метка. 
Для более сложных ситуаций нейросеть может нас и обмануть — мы никак не прописали явно, что в z не должно быть информации о метке. Ну что же, мы можем добавить это требование. А как? Добавим еще нейросеть, которая будет пытаться предсказать метку на основе только вектора z. 

Если эта нейросеть предсказывает хорошо, то енкодер не избавился от информации о метки и мы его за это штрафуем. 

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/disentagle_aae2.png)


## Semisupervised AAE
А еще мы можем кормить в нашу нейросетку неразмеченные объекты! Для них нейросетка сама будет пытаться определить label. Просто не будем штрафовать нейросеть за неверное предсказание метки (мы же ее и не знаем). А кроме них будем кормить и размеченные. Получаем semisupervised learning. 


Чтобы на неразмеченных объектах нейросеть не генерировала мусор, можем потребовать от нее, чтобы метка, которую она генерит приходила из категориального распределения. Как это сделать? Да опять же, добавим дискриминатор, который будет отличать получившееся распределение меток от реального ([категориального](https://en.wikipedia.org/wiki/Categorical_distribution)).

![alttext](https://edunet.kea.su/repo/src/L0X_Encoders/img/semi_aae.png)

## Полезные материалы 

### Про unsupervised learning при помощи нейросетей

Главы из учебника Гудфеллоу по теме:
1. [Representation learning](https://www.deeplearningbook.org/contents/representation.html)
2. [Генеративные модели](https://www.deeplearningbook.org/contents/generative_models.html)

Про все увеличивающуюся роль unsupervised learning: 
[Unsupervised Deep Learning - Google DeepMind & Facebook Artificial Intelligence NeurIPS 2018](https://www.youtube.com/watch?v=rjZCjosEFpI)

[Лекция по генеративным моделям](https://www.youtube.com/watch?v=5WoItGTWV54)

Про проклятье размерности: 
1. [В целом](https://medium.com/diogo-menezes-borges/give-me-the-antidote-for-the-curse-of-dimensionality-b14bce4bf4d2)
2. [Для классификации](https://www.visiondummy.com/2014/04/curse-dimensionality-affect-classification/)
3. [Немного другой взгляд](https://towardsdatascience.com/the-curse-of-dimensionality-f07c66128fe1)

### Автоенкодеры

[Главы из учебника Гудфеллоу по теме](https://www.deeplearningbook.org/contents/autoencoders.html)



[Более подробно про PCA и ссылка на его применение для MNIST](https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60)

[Способы понижения размерности, PCA и разные типы автокодировщиков, лекция Техносферы](https://www.youtube.com/watch?v=W5JLSKcuaQo)

[Eigenfaces](https://ieeexplore.ieee.org/document/139758)

[Конспект от Andrew Ng по разреженным автокодировщикам](https://web.stanford.edu/class/cs294a/sparseAutoencoder.pdf)

Получение разреженного автоенкодера при помощи:

1. [L1-loss а](https://debuggercafe.com/sparse-autoencoders-using-l1-regularization-with-pytorch/)

2. [KL-divergence](https://debuggercafe.com/sparse-autoencoders-using-kl-divergence-with-pytorch/)

Удаление шума из
1. [изображений](https://debuggercafe.com/autoencoder-neural-network-application-to-image-denoising/)
2. [текста](https://debuggercafe.com/denoising-text-image-documents-using-autoencoders/)

[Введение в автоенкодеры на kaggle](https://www.kaggle.com/shivamb/how-autoencoders-work-intro-and-usecases)

### Вариационные автоенкодеры 

[Введение в автоенкодеры, вариационные автоенкодеры, PCA](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73)

Введение в автоенкодеры на Хабрахабре
1. [Введение](https://habr.com/ru/post/331382/)
2. [Manifold learning и скрытые (latent) переменные](https://habr.com/ru/post/331500/)
3. [Вариационные автоэнкодеры (VAE)](https://habr.com/ru/post/331552/)
4. [Conditional VAE](https://habr.com/ru/post/331664/)
5. [GAN(Generative Adversarial Networks)](https://habr.com/ru/post/332000/)
6. [GAN + VAE](https://habr.com/ru/post/332074/)


[Оригинальная статья по VAE](https://arxiv.org/abs/1312.6114)

[Ali Ghodsi, Лекция по VAE](https://www.youtube.com/watch?v=uaaqyVS9-rM)
[Irhum Shafkat, Введение в автоенкодеры, векторная арифметика](https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf)

[Jeremy Jordan, введение в автоенкодеры](https://www.jeremyjordan.me/autoencoders/)
[Jeremy Jordan, вариационные автоенкодеры](https://www.jeremyjordan.me/variational-autoencoders/)

[Туториал по VAE с arxiv](https://arxiv.org/pdf/1606.05908.pdf)

[Еще одно введение в вариационные автоенкодеры](https://livebook.manning.com/book/deep-learning-with-python/chapter-8/)

[Туториал по VAE от Google по tensorflow](https://www.tensorflow.org/tutorials/generative/cvae)

[Векторная арифметика в VAE при генерации изображений](https://blog.otoro.net/2016/04/01/generating-large-images-from-latent-vectors/)

[Генерация анимированных персонажей](https://mlexplained.wordpress.com/category/generative-models/vae/)

[Генерация лиц, можно менять пол, заставлять знаменитостей улыбаться](https://towardsdatascience.com/variational-autoencoders-vaes-for-dummies-step-by-step-tutorial-69e6d1c9d8e9)

[VAE на pytorch с пояснениями](https://debuggercafe.com/getting-started-with-variational-autoencoder-using-pytorch/)


[Введение в условные вариационные автоенкодеры](https://ijdykeman.github.io/ml/2016/12/21/cvae.html)

[Репозиторий с различными модификациями вариационных автоенкодеров](https://github.com/AntixK/PyTorch-VAE)


Взгляд на VAE как на игру с двумя участниками:
1. [часть 1](https://towardsdatascience.com/the-variational-autoencoder-as-a-two-player-game-part-i-4c3737f0987b)
2. [часть 2](https://towardsdatascience.com/the-variational-autoencoder-as-a-two-player-game-part-ii-b80d48512f46)
3. [часть 3](https://towardsdatascience.com/the-variational-autoencoder-as-a-two-player-game-part-iii-d8d56c301600)


### KL-дивергенция 

[Википедия по дивергенции Кульбака-Лейблера](https://ru.wikipedia.org/wiki/Расстояние_Кульбака_—_Лейблера)
[Мотивация KL-дивергенции](https://math.stackexchange.com/questions/90537/what-is-the-motivation-of-the-kullback-leibler-divergence)

Объяснение проблем и разницы между KL-дивергенцией, дивергенцией Йенсена-Шеннона и расстоянием Вассерштейна:
1. [часть 1, проблемы KL-дивергенции. Дивергенция Йенсена-Шеннона](https://www.youtube.com/watch?v=_z9bdayg8ZI) и 
2. [часть 2, проблемы KL-дивергенции и дивергенции Йенсена-Шеннона. Расстояние Вассерштейна](https://www.youtube.com/watch?v=y8LGAhzCOxQ)


### AAE 

Цикл статей по AAE:
1. [Автоенкодеры](https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-1-autoencoder-d9a5f8795af4)
2. [Добавление дискриминатора](https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-2-exploring-latent-space-with-adversarial-2d53a6f8a4f9)
3. [Разделение стиля и содержания при помощи AAE](https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-3-disentanglement-of-style-and-content-89262973a4d7)
4. [Semisupervised learning при помощи AAE](https://towardsdatascience.com/a-wizards-guide-to-adversarial-autoencoders-part-4-classify-mnist-using-1000-labels-2ca08071f95)

[Примеры AAE на tensorflow](https://github.com/nicklhy/AdversarialAutoEncoder)

[Здесь в 6 и 8 лекции тоже можно найти примеры](https://github.com/che-shr-cat/deep-learning-for-biology-hse-2019-course)

### Модификации автоенкодеров

[Contractive Autoencoders](http://www.icml-2011.org/papers/455_icmlpaper.pdf) - автоенкодеры, родственные denoising autoencoders

[Variational losssy autoencoder](https://arxiv.org/pdf/1611.02731.pdf) - один из типов VAE, который пытается решить проблему того, что сильные декодер может игнорировать латентное представление. 

[$\beta$- VAE](https://arxiv.org/pdf/1804.03599.pdf) - еще одно возможное улучшение VAE

[Wassershtein autoencoders](https://arxiv.org/pdf/1711.01558.pdf)

[Concrete autoencoders](https://arxiv.org/abs/1901.09346) - якобы позволяют выделять наиболее важные признаки.

## Примеры практического применения 


1. [Age Progression/Regression - преедсказание того, как будет выглядеть человек в другом возрасте](https:/arxiv/abs/1702.08423)

2. [druGAN, генерация новых химических веществ](https://pubs.acs.org/doi/10.1021/acs.molpharmaceut.7b00346)

3. [Генерация лекарств, специфически меняющих активность генов человека](https://www.frontiersin.org/articles/10.3389/fphar.2020.00269/full)

4. [Генерация ингибиторов определенного белка](https://www.nature.com/articles/s41587-019-0224-x)

5. [Получение латентных представлений транскриптомов](https://academic.oup.com/nar/article/48/10/e56/5814052)

6. [MethylNet](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-020-3443-8) - Использование метилирования генома для обучения латентного представления, помогающего в предсказании возраста и т.д

7. [scVAE](https://academic.oup.com/bioinformatics/article-abstract/36/16/4415/5838187?redirectedFrom=fulltext) - получение данных об экспрессии генов из single cell данных